In [1]:
import pandas as pd
from pandarallel import pandarallel

pos_news_df = pd.read_parquet('pos_topics.parquet')
neg_news_df = pd.read_parquet('neg_topics.parquet')

In [4]:
%%time

import re

pos_news_df['cleaned_text'] = pos_news_df['text'].apply(lambda x: re.sub(r'(?:\@|http?\://|https?\://|www)\S+','',str(x)))
pos_news_df['cleaned_text'] = pos_news_df['cleaned_text'].apply(lambda x: re.sub(r'(?:\n|\r|\t)','', str(x)))
pos_news_df['cleaned_text'] = pos_news_df['cleaned_text'].map(lambda x: re.sub(r'\B#\w*[a-zA-Z]+\w*','',str(x)))

CPU times: user 21.1 s, sys: 1.38 s, total: 22.5 s
Wall time: 22.5 s


In [7]:
pos_news_df

,url,date,language,title,text,cleaned_text,Positive Probability,Negative Probability,Sentiment,Topic Number
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,Artificial intelligence improves parking effic...,0.337937,6.620632e-01,Positive,16
3,http://www.homeoffice.consumerelectronicsnet.c...,2021-03-10,en,Strategy Analytics: 71% of Smartphones Sold Gl...,\n\nStrategy Analytics: 71% of Smartphones Sol...,Strategy Analytics: 71% of Smartphones Sold Gl...,0.347064,6.529362e-01,Positive,-1
13,https://9to5mac.com/2021/03/25/apple-bought-mo...,2021-03-26,en,Apple bought more AI companies than anyone els...,\nApple bought more AI companies than anyone e...,Apple bought more AI companies than anyone els...,0.994807,5.193062e-03,Positive,35
20,https://aindustryreports.com/2020/05/01/artifi...,2020-05-01,en,Artificial Intelligence in Video Games Market ...,\n\nArtificial Intelligence in Video Games Mar...,Artificial Intelligence in Video Games Market ...,1.000000,4.397073e-08,Positive,39
21,https://amarketresearchgazette.com/artificial-...,2020-04-25,en,Artificial Intelligence in Medicine Market – T...,\n\nArtificial Intelligence in Medicine Market...,Artificial Intelligence in Medicine Market – T...,0.999244,7.559952e-04,Positive,5
...,...,...,...,...,...,...,...,...,...,...
200315,https://www.wvlt.tv/prnewswire/2023/04/27/how-...,2023-04-27,en,How To Prepare For The AI Innovation Revolutio...,How To Prepare For The AI Innovation Revolutio...,How To Prepare For The AI Innovation Revolutio...,0.487624,5.123764e-01,Positive,0
200321,https://www.wymt.com/prnewswire/2021/11/04/ime...,2021-11-04,en,"iMerit Launches New Self Serve API Tool, Allow...","iMerit Launches New Self Serve API Tool, Allow...","iMerit Launches New Self Serve API Tool, Allow...",0.999033,9.670553e-04,Positive,86
200322,https://www.wymt.com/prnewswire/2021/11/17/ele...,2021-11-17,en,ElectrifAi Showcases Pre-Structured Machine Le...,ElectrifAi Showcases Pre-Structured Machine Le...,ElectrifAi Showcases Pre-Structured Machine Le...,1.000000,6.763632e-08,Positive,-1
200329,https://www.zdnet.com/article/from-data-to-kno...,2020-12-22,en,From data to knowledge and AI via graphs: Tech...,\n\nFrom data to knowledge and AI via graphs: ...,From data to knowledge and AI via graphs: Tech...,1.000000,0.000000e+00,Positive,88


In [5]:
import spacy
#!python -m spacy download en_core_web_sm
#!pip install en_core_web_sm
nlp = spacy.load("en_core_web_sm")

2023-05-25 16:40:44.911071: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 16:40:45.737977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
def extract_entities_text(text):
    doc = nlp(text)
    entities = []
    labels = []
    for ent in doc.ents:
        entities.append(ent.text)
        labels.append(ent.label_)
    return entities,labels

In [8]:
import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [9]:
%time results = pos_news_df["cleaned_text"].parallel_apply(extract_entities_text)

CPU times: user 46.9 s, sys: 15.5 s, total: 1min 2s
Wall time: 4min 18s


In [11]:
entities = results.apply(lambda x: x[0])
labels = results.apply(lambda x: x[1])

In [14]:
# Flatten the entities and labels lists
flattened_entities = [entity for sublist in entities for entity in sublist]
flattened_labels = [label for sublist in labels for label in sublist]

# Create the DataFrame
pos_ner_df = pd.DataFrame({'entities': flattened_entities, 'labels': flattened_labels})

In [15]:
pos_ner_df

,entities,labels
0,Chinese,NORP
1,ChinaBusinessMilitaryWorldSocietyCultureTravel...,DATE
2,2021Home>>Artificial,CARDINAL
3,Chinese,NORP
4,People's Daily,ORG
...,...,...
6274886,FridayTechie,ORG
6274887,InvestmentsResourcesExpert OpinionAboutAdvertise,PERSON
6274888,CenterContact UsWork,PERSON
6274889,2020,DATE


In [23]:
pos_ner_df.to_parquet("pos_ner.parquet")
pos_news_df.to_parquet("pos_topics.parquet")

In [16]:
%%time

neg_news_df['cleaned_text'] = neg_news_df['text'].apply(lambda x: re.sub(r'(?:\@|http?\://|https?\://|www)\S+','',str(x)))
neg_news_df['cleaned_text'] = neg_news_df['cleaned_text'].apply(lambda x: re.sub(r'(?:\n|\r|\t)','', str(x)))
neg_news_df['cleaned_text'] = neg_news_df['cleaned_text'].map(lambda x: re.sub(r'\B#\w*[a-zA-Z]+\w*','',str(x)))

CPU times: user 33.4 s, sys: 2.18 s, total: 35.6 s
Wall time: 35.6 s


In [17]:
neg_news_df

,url,date,language,title,text,cleaned_text,Positive Probability,Negative Probability,Sentiment,Topic Number
14,https://abcnews.go.com/Nightline/video/trump-d...,2023-04-06,en,Video Trump deepfakes on social media prompt w...,\n\nVideo Trump deepfakes on social media prom...,Video Trump deepfakes on social media prompt w...,1.094404e-05,0.999989,Negative,-1
15,https://abcnews.go.com/Technology/video/eu-imp...,2020-02-19,en,EU imposes sweeping regulations on facial reco...,\n\nEU imposes sweeping regulations on facial ...,EU imposes sweeping regulations on facial reco...,5.047639e-05,0.999950,Negative,87
16,https://africa.com/calling-all-startups-in-the...,2021-04-03,en,Calling All Startups in the AI Space - Africa.com,\nCalling All Startups in the AI Space - Afric...,Calling All Startups in the AI Space - Africa....,1.308936e-20,1.000000,Negative,19
19,https://agenparl.eu/funding-boost-for-artifici...,2020-08-28,en,Funding boost for artificial intelligence in N...,\nFunding boost for artificial intelligence in...,Funding boost for artificial intelligence in N...,1.012077e-04,0.999899,Negative,-1
23,https://arstechnica.com/tech-policy/2020/05/wa...,2020-05-31,en,Walmart employees are out to show its anti-sho...,\n\nWalmart employees are out to show its anti...,Walmart employees are out to show its anti-sho...,1.053579e-44,1.000000,Negative,-1
...,...,...,...,...,...,...,...,...,...,...
200318,https://www.wwnytv.com/prnewswire/2022/02/22/l...,2022-02-22,en,"LXT Introduces New Executive Survey, The Path ...","LXT Introduces New Executive Survey, The Path ...","LXT Introduces New Executive Survey, The Path ...",1.367453e-05,0.999986,Negative,-1
200323,https://www.wymt.com/prnewswire/2022/03/16/usi...,2022-03-16,en,Using Artificial Intelligence to Solve One of ...,Using Artificial Intelligence to Solve One of ...,Using Artificial Intelligence to Solve One of ...,3.381574e-10,1.000000,Negative,-1
200324,https://www.wymt.com/prnewswire/2023/03/01/dri...,2023-03-01,en,"Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...","Drift Introduces GPT Integration, Advancing AI...",1.679468e-05,0.999983,Negative,18
200325,https://www.wymt.com/prnewswire/2023/04/28/uro...,2023-04-28,en,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,Urologists Explore AI for Prostate Cancer Dete...,2.481816e-03,0.997518,Negative,-1


In [18]:
%time results = neg_news_df["cleaned_text"].parallel_apply(extract_entities_text)

CPU times: user 1min 10s, sys: 24 s, total: 1min 34s
Wall time: 6min 34s


In [19]:
entities = results.apply(lambda x: x[0])
labels = results.apply(lambda x: x[1])

In [20]:
# Flatten the entities and labels lists
flattened_entities = [entity for sublist in entities for entity in sublist]
flattened_labels = [label for sublist in labels for label in sublist]

# Create the DataFrame
neg_ner_df = pd.DataFrame({'entities': flattened_entities, 'labels': flattened_labels})

In [21]:
neg_ner_df

,entities,labels
0,AI,ORG
1,AmericaInterest Successfully,ORG
2,INStream,ORG
3,Jan. 6,DATE
4,COVID,ORG
...,...,...
9401509,Newsletters Mobile,PERSON
9401510,Apps Facebook,PERSON
9401511,eNewspaper Archives,ORG
9401512,Homes Classifieds,PERSON


In [24]:
neg_ner_df.to_parquet("neg_ner.parquet")
neg_news_df.to_parquet("neg_topics.parquet")